# How to use the UpSampling2D and Conv2DTranspose Layers in Keras
(https://machinelearningmastery.com/upsampling-and-transpose-convolution-layers-for-generative-adversarial-networks/)

In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Sequential
from keras.layers import UpSampling2D
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Reshape
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv2DTranspose
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### How to Use the UpSampling2D Layer

In [6]:
...
# define model
model = Sequential()
model.add(UpSampling2D())

In [8]:
...
# define input data
X = np.asarray([[1, 2],
			 [3, 4]])
# show input data for context
print(X)

[[1 2]
 [3 4]]


In [9]:
...
# reshape input data into one sample a sample with a channel
X = X.reshape((1, 2, 2, 1))

In [10]:
...
# define model
model = Sequential()
model.add(UpSampling2D(input_shape=(2, 2, 1)))
# summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_1 (UpSampling2 (None, 4, 4, 1)           0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [11]:
...
# make a prediction with the model
yhat = model.predict(X)

In [12]:
...
# reshape output to remove channel to make printing easier
yhat = yhat.reshape((4, 4))
# summarize output
print(yhat)

[[1. 1. 2. 2.]
 [1. 1. 2. 2.]
 [3. 3. 4. 4.]
 [3. 3. 4. 4.]]


In [13]:
...
# example of using different scale factors for each dimension
model.add(UpSampling2D(size=(2, 3)))

In [14]:
...
# example of using bilinear interpolation when upsampling
model.add(UpSampling2D(interpolation='bilinear'))

### Simple Generator Model With the UpSampling2D Layer
- GAN generator model must produce a 10×10 image and take a 100 element vector from the latent space as input.
- a Dense fully connected layer can be used to interpret the input vector and create a sufficient number of activations (outputs) that can be reshaped into a low-resolution version of our output image, in this case, 128 versions of a 5×5 image.

In [19]:
...
# define model
model = Sequential()
# define input shape, output enough activations for for 128 5x5 image
model.add(Dense(128 * 5 * 5, input_dim=100))
# reshape vector of activations into 128 feature maps with 5x5
model.add(Reshape((5, 5, 128)))

In [21]:
...
# double input from 128 5x5 to 1 10x10 feature map
model.add(UpSampling2D())

In [24]:
...
# fill in detail in the upsampled feature maps
model.add(Conv2D(1, (3,3), padding='same'))

In [25]:
# summarize model
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3200)              323200    
_________________________________________________________________
reshape (Reshape)            (None, 5, 5, 128)         0         
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 10, 10, 128)       0         
_________________________________________________________________
up_sampling2d_5 (UpSampling2 (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 1)         1153      
Total params: 324,353
Trainable params: 324,353
Non-trainable params: 0
_________________________________________________________________


### How to Use the Conv2DTranspose Layer

In [30]:
...
# define input data
X = np.asarray([[1, 2],
			 [3, 4]])
# show input data for context
print(X)

[[1 2]
 [3 4]]


In [31]:
...
# reshape input data into one sample a sample with a channel
X = X.reshape((1, 2, 2, 1))

In [32]:
...
# define model
model = Sequential()
model.add(Conv2DTranspose(1, (1,1), strides=(2,2), input_shape=(2, 2, 1)))
# summarize the model
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose (Conv2DTran (None, 4, 4, 1)           2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [34]:
...
# define weights that they do nothing
weights = [np.asarray([[[[1]]]]), np.asarray([0])]
# store the weights in the model
model.set_weights(weights)

In [35]:
...
# make a prediction with the model
yhat = model.predict(X)

In [36]:
...
# reshape output to remove channel to make printing easier
yhat = yhat.reshape((4, 4))
# summarize output
print(yhat)

[[1. 0. 2. 0.]
 [0. 0. 0. 0.]
 [3. 0. 4. 0.]
 [0. 0. 0. 0.]]


In [37]:
...
# example of using padding to ensure that the output is only doubled
model.add(Conv2DTranspose(1, (3,3), strides=(2,2), padding='same', input_shape=(2, 2, 1)))

### Simple Generator Model With the Conv2DTranspose Layer

In [38]:
...
# define model
model = Sequential()
# define input shape, output enough activations for for 128 5x5 image
model.add(Dense(128 * 5 * 5, input_dim=100))
# reshape vector of activations into 128 feature maps with 5x5
model.add(Reshape((5, 5, 128)))

In [39]:
...
# double input from 128 5x5 to 1 10x10 feature map
model.add(Conv2DTranspose(1, (3,3), strides=(2,2), padding='same'))

In [ ]:
# summarize model
model.summary()